In [4]:
import spacy
import os
import pandas as pd
from spacy.matcher import Matcher

In [3]:
Condition = ['depending', 'necessary', 'appropriate',
                 'inappropriate', 'as needed', 'as applicable', 'when applicable', 'if applicable',
                   'where applicable', 'when needed', 'if needed', 'where needed' 'reasonable', 'reasonably'
                 'otherwise reasonably', 'sometimes',
                 'from time to time']
Generalization = ['generally', 'mostly', 'widely',
                  'general', 'commonly',
                  'usually', 'normally', 'typically',
                  'largely', 'often', 'primarily',
                  'among other things']
Modality = ['may', 'might', 'can', 'could', 'would',
            'likely', 'possible', 'possibly']
Numeric_quantifier = ['anyone', 'certain', 'everyone',
                      'numerous', 'some', 'most', 'few',
                      'much', 'many', 'various', 'include',
                      'limited to', 
                      'including but not limited to']

In [6]:
Path = "./Privacy_Policies/"
filelist = os.listdir(Path)
txt_files = []
file_names = []
for i in filelist:
    if i.endswith(".txt"):
        file_names.append(i)
        with open(Path + i, 'r') as f:
            txt_files.append(f.read())

## Policy String

In [21]:
Policies = pd.DataFrame([file_names, txt_files]).T
Policies.columns = ["File_name", "text_content"]

In [22]:
Policies

,File_name,text_content
0,FLStudioMobile.txt,MOBILE APP PRIVACY | FL Studio FL StudioOvervi...
1,TouchRetouch.txt,ADVA Soft - Privacy PolicyToggle navigationCon...
2,SnackyTakatak.txt,Privacy Policy – MX TakaTakMX TakaTak Privacy ...
3,MXTakaTakShortVideoApp|MadeinIndiaforYou.txt,Privacy Policy – MX TakaTakMX TakaTak Privacy ...
4,Snapchat.txt,Privacy Policy - Snap Inc.HomeCareersNewsInves...
...,...,...
73,PGT+🔧:ProGFX&Optimizer(withadvancesetting).txt,Privacy PolicyYour privacy is important to us....
74,QuickHealTotalSecurity.txt,PRIVACY POLICY- QUICK HEAL MOBILE DEVICE SECUR...
75,"MXPlayer:VideoPlayer,Movies,Songs&GamesApp.txt",MX Player Privacy PolicySearch this siteHomeFe...
76,InternetOptimizerPro|No-Ads.txt,Privacy PolicyPrivacy Policy ...


In [23]:
string_in = Policies.text_content[0]

## New Heading


In [26]:
nlp = spacy.load('en')

def generateMatrix(text_string):
    tok = nlp(text_string)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.